In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

In [2]:
# Инициализируем модель и токенизатор

model_name = "cointegrated/rubert-tiny"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Загружаем данные

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/valid.csv')

print(train.shape)
print(test.shape)

(189891, 3)
(21098, 3)


In [4]:
# Проверяем наличие пустых строк

print(f'Train:\n{train.isna().sum()}')
print()
print(f'Test: \n{test.isna().sum()}')

Train:
Unnamed: 0    0
text          0
sentiment     0
dtype: int64

Test: 
Unnamed: 0    0
text          0
sentiment     0
dtype: int64


In [5]:
# Создаем словарь, в котором число соответсвует сентименту сообщения

sentiment_dict = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

In [6]:
# Преобразуем данные в подходящий для модели формат

train_set = Dataset.from_pandas(train)
test_set = Dataset.from_pandas(test)

In [7]:
# Создадим функцию для токенизации текстов, токенизируем данные

def tokenize(batch):
    return tokenizer(batch["text"], max_length=256, padding='max_length', truncation=True)

train = train_set.map(tokenize, batched=True)
test = test_set.map(tokenize, batched=True)

Map:   0%|          | 0/189891 [00:00<?, ? examples/s]

Map:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [8]:
train

Dataset({
    features: ['Unnamed: 0', 'text', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 189891
})

In [9]:
# Переименуем целевую переменную
train = train.rename_column("sentiment", "labels")
test = test.rename_column("sentiment", "labels")

# Удалим ненужные признаки
train = train.remove_columns(["Unnamed: 0", "text"])
test = test.remove_columns(["Unnamed: 0", "text"])

In [10]:
train

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 189891
})

In [11]:
# Функция для отображения метрик качества

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


# Так же добавим в конфигурацию словарь соответсвия сентиментов для человекочитаемого отображения

model.config.id2label = sentiment_dict
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [12]:
args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True
)

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-13-b311bde07710>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pavelzel95 (pavelzel95-self-employed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.509700,0.510793,0.764243,0.760483


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.509700,0.510793,0.764243,0.760483
2,0.475000,0.493918,0.774671,0.773882


TrainOutput(global_step=23738, training_loss=0.533873377497952, metrics={'train_runtime': 69684.0402, 'train_samples_per_second': 5.45, 'train_steps_per_second': 0.341, 'total_flos': 1400478123912192.0, 'train_loss': 0.533873377497952, 'epoch': 2.0})

In [15]:
model.save_pretrained("model/")
tokenizer.save_pretrained("model/")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')